In [1]:
import SciServer
#from SciServer import Authentication, LoginPortal, Config, CasJobs, SkyQuery, SciDrive, SkyServer
from SciServer import CasJobs
#import os
#import pandas
#import sys
#import json
#from io import StringIO
#from io import BytesIO
#from PIL import Image
#import numpy as np
#import matplotlib.pyplot as plt
print('Imported SciServer modules')

Imported SciServer modules


In [2]:
import pandas
import numpy as np
from pprint import pprint
from datetime import datetime, timedelta
print('Imported other needed modules')

Imported other needed modules


In [ ]:
# Read the help document for the entire SciServer package
help(SciServer)

In [ ]:
# Read the help document for the CasJobs module
help(CasJobs)

In [7]:
# Define some very simple test cases for the purposes of this example notebook

CasJobs_TestDatabase = "MyDB" # use your MyDB - normally you won't need to change this
CasJobs_TestQuery = "select 4 as Column1, 5 as Column2 " # toy query, simply returns two columns of constants: 4, 5.
CasJobs_TestTableName1 = "MyNewtable1"    # the name of a new table to be created (doesn't yet exist)
CasJobs_TestTableName2 = "MyNewtable2"    # the name of a new table to be created (doesn't yet exist)
CasJobs_TestTableCSV = u"Column1,Column2\n6,7\n"    # toy CSV file: 1 row x 2 columns of constants: 6, 7.
CasJobs_TestCSVFile = "SciScriptTestFile.csv"       # a more real CSV file. It must be included in this dir in current container.
CasJobs_TestFitsFile = "SciScriptTestFile.fits"     # a FITS file to upload


print('Test case variables set.')

Test case variables set.


In [4]:
#get user schema info

casJobsId = CasJobs.getSchemaName()
print(casJobsId)

wsid_1220987707


In [5]:
# Returns the following information abaout the tables in your MyDB (as a Python dictionary object):
### Size: size of the table (in kB)
### Name: the name of the table
### Rows: the number of rows the table contains
### Date: the date of the table's creation, as the number of 10-microsecond intervals elapsed 1 AD

# These convenience functions will print your list of tables in human-readable format

def better_createdate(cjCreateDate):     # returns table create date as a datetime object
    createsec = cjCreateDate / 10000000  # Divide by 10 million to get seconds elapsed since 1 AD
    firstday = datetime(1, 1, 1, 0, 0)   # Save 1 AD as "firstday"
    created = firstday + timedelta(seconds=createsec)  # Get calendar date on which table was created
    return (created)

def tables_formatted(tables):   # better formatted printing of a tables dictionary (output of get_tables)
    for thistable in tables:
        print('Table name: ',thistable['Name'])
        print('Rows: ',thistable['Rows'])
        print('Size (kB): ',thistable['Size'])
        betterdate = better_createdate(thistable['Date'])
        print('Created time: ',betterdate.strftime('%Y-%m-%d %H:%M:%S'))
        print('\n')

tables = CasJobs.getTables(context=CasJobs_TestDatabase)
print('Tables in '+CasJobs_TestDatabase+':\n')
tables_formatted(tables)

Tables in MyDB:

Table name:  MyNewtable1
Rows:  1
Size (kB):  16
Created time:  2017-08-02 09:59:38


Table name:  MyNewtable2
Rows:  1
Size (kB):  16
Created time:  2017-08-02 10:15:54


Table name:  thing1
Rows:  1
Size (kB):  16
Created time:  2017-08-02 10:20:30


Table name:  thing2
Rows:  1
Size (kB):  16
Created time:  2017-08-02 10:21:00


Table name:  thing3
Rows:  1
Size (kB):  16
Created time:  2017-08-02 10:29:22




In [8]:
#executes a quick SQL query and store results in a pandas dataframe:


df = CasJobs.executeQuery(sql=CasJobs_TestQuery, context=CasJobs_TestDatabase, format="pandas")
df

# Other options for return format (format=...):
# 'json': a JSON string containing the query results
# 'dict': a dictionary created from the JSON string containing the query results
# 'csv': a csv string
# 'readable': an object of type io.StringIO, which has the .read() method and wraps a csv string that can be passed into pandas.read_csv for example
# 'StringIO': an object of type io.StringIO, which has the .read() method and wraps a csv string that can be passed into pandas.read_csv for example
# 'fits': an object of type io.BytesIO, which has the .read() method and wraps the result in fits format
# 'BytesIO': an object of type io.BytesIO, which has the .read() method and wraps the result in fits format

# CasJobs.executeQuery(sql, context, format)

,Column1,Column2
0,4,5


In [10]:
#submit a job, which inserts the query results into a table in the MyDB database context. 
#Wait until the job is done and get its status.
#myquery = CasJobs_TestQuerySelect + '\n'
#myquery += 'into' + CasJobs_TestDatabase + '.' + CasJobs_TestTableName2

print('Query to be executed: ',CasJobs_TestQuery,'\n')

jobId = CasJobs.submitJob(sql=CasJobs_TestQuery + " into MyDB." + CasJobs_TestTableName2, context="MyDB")
print('Job submitted with jobId = ',jobId)
jobDescription = CasJobs.waitForJob(jobId=jobId, verbose=True)
print('\n')
print('Information about the job:')
pprint(jobDescription)

Query to be executed:  select 4 as Column1, 5 as Column2  

Job submitted with jobId =  29953162
Waiting...Waiting...Waiting...Waiting...Done!


Information about the job:
{'AutoComplete': 0,
 'Created_Table': 'MyNewtable2',
 'Estimate': 500,
 'HostIP': 'SQL001DBHost   ',
 'JobID': 29953162,
 'Message': 'Query Complete',
 'ModifiedQuery': 'select 4 as Column1, 5 as Column2  '
                  '/*CASJOBS_INTO_TABLE:[sdss3q].mydb_1220987707.webuser.MyNewtable2*/',
 'OutputLoc': '',
 'OutputType': 'QUERY',
 'Params': '',
 'Query': 'select 4 as Column1, 5 as Column2  into MyDB.MyNewtable2',
 'Rows': 1,
 'SendEmail': 0,
 'Status': 5,
 'Target': 'MyDB',
 'TaskName': 'Compute.SciScript-Python.CasJobs.submitJob',
 'TimeEnd': '2017-08-02T13:28:25.493',
 'TimeStart': '2017-08-02T13:28:25.367',
 'TimeSubmit': '2017-08-02T13:28:21.06',
 'WebServicesID': 1220987707}


In [11]:
bigtablename = 'hugetable'
verylongquery = 'select top 5000000 objid,ra,dec,u,g,r,i,z from dr14.photoobjall'

#print(verylongquery)
print(verylongquery +"  into MyDB." + bigtablename)

jobId = CasJobs.submitJob(sql=verylongquery +"  into MyDB." + bigtablename, context="dr14")
print('Job submitted with jobId = ',jobId)
jobDescription = CasJobs.waitForJob(jobId=jobId, verbose=True)
print('\n')
print('Information about the job:')
pprint(jobDescription)

#print('AutoComplete: ',jobDescription['AutoComplete'])
#print('Created_Table: ',jobDescription['Created_Table'])
#print('JobID: ',jobDescription['JobID'])
#print('Message: ',jobDescription['Message'])
#print('Rows: ',jobDescription['Rows'])
#print('Status: ',jobDescription['Status'])
#print('TaskName: ',jobDescription['TaskName'])
#print('TimeSubmit: ',jobDescription['TimeSubmit'])
#print('TimeStart: ',jobDescription['TimeStart'])
#print('TimeEnd: ',jobDescription['TimeEnd'])
duration = pandas.to_datetime(jobDescription['TimeEnd']) - pandas.to_datetime(jobDescription['TimeStart'])
print('Query duration: ',duration.seconds, 'seconds')

select top 5000000 objid,ra,dec,u,g,r,i,z from dr14.photoobjall  into MyDB.hugetable
Job submitted with jobId =  29953164
Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting...Waiting.

In [ ]:
print('2334243l2jkl423')

In [ ]:
jobDescription = CasJobs.getJobStatus(29952359)